# Backup Workspace
## Author: Jonn Smith
## Date: 2023/02/28
Detect all workspace data and tables, then download and back them up.
***

In [1]:
import time
import os
import datetime
import gzip
import io
import json

import pandas as pd
import firecloud.api as fapi
import numpy as np

from google.cloud import bigquery
from google.cloud import storage
from google.api_core.exceptions import NotFound

import lrmaCU.terra.table_utils as lrma_table_utils
import lrmaCU.terra.workspace_utils as lrma_workspace_utils

In [2]:
# Track start time so we can calculate elapsed time for backup:
START_TIME = time.time()

## Environment
Set up our environment (Terra namespace, workspace, and the location of the bucket(s)).

In [3]:
namespace = os.environ['WORKSPACE_NAMESPACE']
workspace = os.environ['WORKSPACE_NAME']
default_bucket = os.environ['WORKSPACE_BUCKET']

print(f"Namespace: {namespace}")
print(f"Workspace: {workspace}")
print(f"Default Bucket: {default_bucket}")

Namespace: broad-firecloud-dsde-methods
Workspace: sr-malaria
Default Bucket: gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd


## Get and dump our entities

In [4]:
def load_table(namespace, workspace, table_name, store_membership=False):
    ent_old = fapi.get_entities(namespace, workspace, table_name).json()
    tbl_old = None

    membership = None
    if len(ent_old) > 0:
        tbl_old = pd.DataFrame(list(map(lambda e: e['attributes'], ent_old)))
        tbl_old[f"entity:{table_name}_id"] = list(map(lambda f: f['name'], ent_old))

        if store_membership:
            membership = list(map(lambda g: set(map(lambda h: h['entityName'], g['items'])), tbl_old['samples']))
            del tbl_old['samples']

        c = list(tbl_old.columns)
        c.remove(f"entity:{table_name}_id")
        c = [f"entity:{table_name}_id"] + c
        tbl_old = tbl_old[c]
        tbl_old = tbl_old.astype(str)

    return tbl_old, membership

# Remove any `nan` values in a given dataframe.
# `nan` values are caused by a parsing issue and are artifacts.
def fix_nans(df, quiet=True):
    if not quiet: print("Replacing all `nan` values with empty strings: ")
    for c in df.columns.values:
        nan_types = ("nan", float('nan'))
        has_nan = False
        num_denaned = 0
        for n in nan_types:
            if (sum(df[c] == n) > 0):
                num_denaned += sum(df[c] == n)
                df.loc[df[c] == n, c] = ""
                has_nan = True
        if has_nan and not quiet:
            print(f"\t{c}: {num_denaned}")

    if not quiet: print("Replacing numpy nan values...")
    if not quiet: print("Done.")
    return df.replace(np.nan, "")

def _write_json_file_to_bucket(backup_folder_path, bucket, timestamp, namespace, workspace, json_object, file_base_name):
    # Write our dict to our bucket:
    blob = bucket.blob(f"{backup_folder_path}/{timestamp}_{namespace}_{workspace}_{file_base_name}.json.gz")
    with blob.open('wb') as f:
        f.write(gzip.compress(bytes(json.dumps(json_object, indent=4), 'utf-8')))

In [5]:
# Get our entity types so we know what to dump:
entity_types = fapi.list_entity_types(namespace, workspace).json()

timestamp = datetime.datetime.now().strftime("%Y%m%dT%H%M%S")
workspace_bucket = fapi.get_workspace(namespace, workspace).json()["workspace"]["bucketName"]
backup_folder_path = f"backups/{timestamp}"

# Create our timestamped backup bucket:
storage_client = storage.Client()
bucket = storage_client.bucket(workspace_bucket)

In [6]:
# Iterate over entity types and dump each one to a separate TSV:
print(f"Writing workspace entities to backup dir:")
print(f"gs://{workspace_bucket}/{backup_folder_path}/tables")
for et in entity_types:
    print(f"\t{et}")
    tbl, _ = load_table(namespace, workspace, et)
    tbl = fix_nans(tbl)
    table_name = f"{timestamp}_{namespace}_{workspace}_{et}.tsv.gz"
    
    # Write our table to our bucket:
    blob = bucket.blob(f"{backup_folder_path}/tables/{table_name}")
    
    with io.StringIO() as buf:
        tbl.to_csv(buf, sep="\t", index=False)
        with blob.open('wb') as f:
            f.write(gzip.compress(bytes(buf.getvalue(), 'utf-8')))
print('Done.')

Writing workspace entities to backup dir:
gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/backups/20240604T152727/tables
	sample_set_set
	z_external_pipeline_validation_set
	sample_set
	pfcrosses
	validation_set
	standard_sample_set
	sample
	validation
	z_external_pipeline_validation
	remote_dataset
	high_quality_assembly
	standard_sample
Done.


In [7]:
# Now backup the notebooks:
print("Writing notebooks to backup dir:")
print(f"gs://{workspace_bucket}/{backup_folder_path}/notebooks")
for notebook_blob in storage_client.list_blobs(workspace_bucket, prefix='notebooks'):
    original_name = notebook_blob.name[notebook_blob.name.find("/")+1:]
    print(f"\t{original_name}")
    notebook_name = f"{timestamp}_{namespace}_{workspace}_{original_name}" 
    blob = bucket.copy_blob(notebook_blob, bucket, new_name=f"{backup_folder_path}/notebooks/{notebook_name}")
print("Done.")

Writing notebooks to backup dir:
gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/backups/20240604T152727/notebooks
	00_backup_workspace.ipynb
	01_import_new_flowcell_data_from_tsv.ipynb
	02_add_new_flowcells_to_sample_table.ipynb
	10_hail_pca_analysis.ipynb
	98_rename_samples.ipynb
	99_Joint_Call_QC_And_Exploration.ipynb
	99_Submission_Statistics.ipynb
	99_fix_sample_table_bam_entries.ipynb
	ZZ_99_workspace_scratch_notebook.ipynb
	bknight.ipynb
	drug_resistance_heatmap.ipynb
	hail_playground.ipynb
	inspect_joint_call_cohort_with_IGV.ipynb
	inspect_sample_with_IGV.ipynb
	kvg_quick_data_review.ipynb
	reduceVCF.ipynb
	sz_00_backup_workspace.ipynb
	sz_base_template_for_all_routines_and_analysis.ipynb
	sz_copy_bai_to_data_table.ipynb
	sz_modify_data_table.ipynb
	sz_rename_samples.ipynb
	sz_using_beris_resource_monitoring.ipynb
	update_sample_table.ipynb
Done.


In [8]:
# Now store workspace attributes:
ws_dict = lrma_workspace_utils._query_workspace(namespace, workspace, 2)
ws_attributes_dict = ws_dict['attributes']

# Write our dict to our bucket:
print(f"Writing workspace attributes to backup dir:")
print(f"gs://{workspace_bucket}/{backup_folder_path}")

_write_json_file_to_bucket(f"{backup_folder_path}", bucket, timestamp, namespace, workspace, ws_attributes_dict,
                           f"workspace_attributes")

print(f"Done.")
print()

# and store the workspace metadata:
del ws_dict['attributes']

print(f"Writing workspace metadata to backup dir:")
print(f"gs://{workspace_bucket}/{backup_folder_path}")

# Write our dict to our bucket:
_write_json_file_to_bucket(f"{backup_folder_path}", bucket, timestamp, namespace, workspace, ws_dict,
                           f"workspace_metadata")
    
print(f"Done.")

Writing workspace attributes to backup dir:
gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/backups/20240604T152727
Done.

Writing workspace metadata to backup dir:
gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/backups/20240604T152727
Done.


In [9]:
# Now store workspace method (i.e. workflow) information:
response = lrma_workspace_utils.retry_fiss_api_call('list_workspace_configs', 2, namespace, workspace, True)
workflow_dict = response.json()

# Sort our workflows in alphabetical order:
workflow_dict = sorted(workflow_dict, key=lambda k: k['name'])

print(f"Writing workflow high-level information to backup dir:")
print(f"gs://{workspace_bucket}/{backup_folder_path}")

# Write our dict to our bucket:
_write_json_file_to_bucket(f"{backup_folder_path}", bucket, timestamp, namespace, workspace, workflow_dict,
                               f"workflows")
    
print(f'Done.')

Writing workflow high-level information to backup dir:
gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/backups/20240604T152727
Done.


In [10]:
# Get the metadata, default inputs, and default outputs for each workflow:

print(f"Writing workflow metadata, inputs, and outputs to backup dir:")
print(f"gs://{workspace_bucket}/{backup_folder_path}/workflows")

for workflow_name in [w["name"] for w in workflow_dict]:
    print(f"\t{workflow_name}")
    
    response = lrma_workspace_utils.retry_fiss_api_call('get_workspace_config', 2, namespace, workspace, namespace, workflow_name)
    metadata = response.json()
    
    # Write the workflow metadata:
    _write_json_file_to_bucket(f"{backup_folder_path}/workflows", bucket, timestamp, namespace, workspace, metadata,
                               f"{workflow_name}_workflow_metadata")
    
    inputs = metadata['inputs']
    outputs = metadata['outputs']
    del metadata['inputs']
    del metadata['outputs']
    
    # Write the workflow inputs:
    _write_json_file_to_bucket(f"{backup_folder_path}/workflows", bucket, timestamp, namespace, workspace, inputs, 
                               f"{workflow_name}_workflow_inputs")
    # Write the workflow outputs:
    _write_json_file_to_bucket(f"{backup_folder_path}/workflows", bucket, timestamp, namespace, workspace, outputs,
                               f"{workflow_name}_workflow_outputs")
        
print(f"Done")

Writing workflow metadata, inputs, and outputs to backup dir:
gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/backups/20240604T152727/workflows
	01_SRFlowcell
	01_SRFlowcell_sz_standard_sample_id
	01_SRFlowcell_sz_standard_sample_id_increase_memory
	01_SRFlowcell_sz_standard_sample_id_increase_memory_sz_add_mem_disk_AlignedMetrics
	02_SRWholeGenome
	02_SRWholeGenome_jts_quick_hc_bugfix
	03_SRJointCallGVCFsWithGenomicsDB
	03_SRJointCallGVCFsWithGenomicsDB_jts_quick_hc_fix
	03_SRJointCallGVCFsWithGenomicsDB_sz_add_dsik_srjointgenotyping_from_jts_quick_hc_bugfix
	03_SRJointCallGVCFsWithGenomicsDB_test_Gnarly
	98_SRWholeGenome_Pf_Niare_VETS
	98_SRWholeGenome_Pf_Niare_VQSR
	99_SRJointCallGVCFsWithGenomicsDB_Pf_Niare_VETS
	99_SRJointCallGVCFsWithGenomicsDB_Pf_Niare_VQSR
	BenchmarkVCFs
	CompareVcfBenchmarks
	ConvertToZarrStore
	DownloadFromSRA
	ExpandedDrugResistanceMarkerAggregation
	ExpandedDrugResistanceMarkerExtraction
	ExtractRegionsFromBam
	GenerateMalariaReport
	HMMIBD_01_CLEAN_VCF
	ONTPf

In [11]:
# Finally store submissions:
response = lrma_workspace_utils.retry_fiss_api_call('list_submissions', 2, namespace, workspace)
submissions_dict = response.json()

# Sort our submissions from most recent to least recent:
submissions_dict = sorted(submissions_dict, key=lambda k: k['submissionDate'], reverse=True)

print(f"Writing workspace job history to backup dir:")
print(f"gs://{workspace_bucket}/{backup_folder_path}")

_write_json_file_to_bucket(backup_folder_path, bucket, timestamp, namespace, workspace, submissions_dict, "workspace_job_history")
    
print(f"Done.")

Writing workspace job history to backup dir:
gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/backups/20240604T152727
Done.


In [12]:
# Track end time so we can calculate elapsed time for backup:
END_TIME = time.time()

In [13]:
import pytz
now_utc = datetime.datetime.utcnow()
timezone = pytz.timezone('America/New_York')
now_et = now_utc.astimezone(timezone)
time_string = now_et.strftime("%A %B %d at %H:%M:%S ET")
print(f"Backup completed on {time_string}")
print(f"Backup location: gs://{workspace_bucket}/{backup_folder_path}")
print(f"Elapsed time: {END_TIME-START_TIME:2.2f}s")

Backup completed on Tuesday June 04 at 11:28:34 ET
Backup location: gs://fc-b06e896e-cc1d-4deb-b638-f7b87c3e5dbd/backups/20240604T152727
Elapsed time: 67.39s


***